<a href="https://colab.research.google.com/github/A-Leus/A/blob/master/GBoosting_on_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
from datetime import datetime as dt
import json
from functools import reduce
import tqdm
!pip install catboost
from catboost import CatBoostClassifier
!pip install yfinance
import yfinance as yf
import numpy as np
# !pip install wordcloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from tqdm import tqdm
import bz2
import _pickle as cPickle
import re

     |████████████████████████████████| 66.1MB 59kB/s 
  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=1af041e24416955ed1039cd09a4e1247e7d4030193f45f7b18dd3362c6e2d538
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:

SOURCE_DIR = '/content/drive/My Drive/stock_news_api/snapi_tickersdb'
ROOT_DIR='/content/drive/My Drive/'

In [5]:
file_name_compiled = os.path.join('/content/drive/My Drive/', 'snapi_all_headlines.pkl')
if os.path.isfile(file_name_compiled):
  df = pd.read_pickle(file_name_compiled)
else:
  import os
  import glob
  df = pd.DataFrame()
  file_names = glob.glob(os.path.join(SOURCE_DIR, "*.csv"))

  for file in file_names:
      cdf= pd.DataFrame()
      print(file)
      cdf = pd.read_csv(file, low_memory=False)
      cdf= cdf.drop(df.columns[0:4], axis =1, inplace=False)
      cdf['date'] = pd.to_datetime(cdf['date'],errors='coerce')
      cdf.sort_values(['date'],ascending=False)
      df=df.append(cdf)
      # df.set_index('date')
  df.to_pickle(file_name_compiled)

In [6]:
df.head()

,Unnamed: 0,ticker,news_url,image_url,title,text,sentiment,source,date,type,topics,tickers
0,0.0,JBLU,https://www.cnn.com/2020/09/01/business/airlin...,https://cdn.snapi.dev/images/v1/y/7/2008311102...,The airline industry could shrink by half to s...,The US airline industry needs to shrink its la...,Neutral,CNN Business,2020-09-01 08:14:24-04:00,Article,[],"['AAL', 'DAL', 'JBLU', 'LUV', 'UAL']"
1,0.0,JBLU,https://www.marketwatch.com/story/airline-stoc...,https://cdn.snapi.dev/images/v1/6/i/airline8.jpg,Airline stocks sink as analyst says United's m...,Shares of airline companies traded broadly low...,Negative,Market Watch,2020-08-31 13:58:53-04:00,Article,[],"['AAL', 'DAL', 'JBLU', 'LUV', 'UAL', 'JETS']"
2,0.0,JBLU,https://www.fool.com/investing/2020/08/27/sp-5...,https://cdn.snapi.dev/images/v1/y/z/urlhttps3a...,"S&P 500 Airline Stocks, Cruise Stocks Surge on...","A big change in Fed policy is good for stocks,...",Positive,The Motley Fool,2020-08-27 17:58:00-04:00,Article,[],"['AAL', 'CCL', 'DAL', 'JBLU', 'LUV', 'NCLH', '..."
3,0.0,JBLU,https://www.fool.com/investing/2020/08/27/why-...,https://cdn.snapi.dev/images/v1/b/w/urlhttps3a...,Why Airline Shares Are Higher Today (Aug 27),A COVID quick test could help reassure uncerta...,Positive,The Motley Fool,2020-08-27 13:25:00-04:00,Article,[],"['AAL', 'DAL', 'JBLU', 'LUV', 'SAVE', 'UAL']"
4,0.0,JBLU,https://www.zacks.com/stock/news/1048181/why-i...,https://cdn.snapi.dev/images/v1/0/m/airline7.jpg,Why Is JetBlue (JBLU) Up 12.1% Since Last Earn...,JetBlue (JBLU) reported earnings 30 days ago. ...,Positive,Zacks Investment Research,2020-08-27 11:31:26-04:00,Article,[],['JBLU']


In [7]:
df['source'].value_counts()
# len(df)

Zacks Investment Research        74836
The Motley Fool                  55081
Seeking Alpha                    27430
CNBC Television                  24022
InvestorPlace                    23468
CNBC                             14953
Benzinga                         14305
Reuters                          13498
GlobeNewsWire                    12301
Market Watch                     11051
Forbes                           10509
Investors Business Daily          7210
24/7 Wall Street                  6799
GuruFocus                         4862
TechCrunch                        4652
Business Wire                     4530
PRNewsWire                        4243
ETF Trends                        3747
CNET                              3641
Investopedia                      3401
CNN Business                      2776
Yahoo Finance                     2555
New York Post                     2311
Bloomberg Markets and Finance     2304
Fox Business                      2101
GeekWire                 

In [8]:
df['sentiment'].value_counts()

Positive    166062
Neutral     119948
Negative     63355
Name: sentiment, dtype: int64

In [9]:
# df_p= df[df.sentiment =='Positive']
# len(df_p)

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)

    return " ".join(tokens)

In [12]:
%%time
# TEXT CLEANING
df.text = df.text.apply(lambda x: preprocess(x))

CPU times: user 18 s, sys: 28.7 ms, total: 18 s
Wall time: 18 s


In [13]:
df.head()

,Unnamed: 0,ticker,news_url,image_url,title,text,sentiment,source,date,type,topics,tickers
0,0.0,JBLU,https://www.cnn.com/2020/09/01/business/airlin...,https://cdn.snapi.dev/images/v1/y/7/2008311102...,The airline industry could shrink by half to s...,us airline industry needs shrink labor costs m...,Neutral,CNN Business,2020-09-01 08:14:24-04:00,Article,[],"['AAL', 'DAL', 'JBLU', 'LUV', 'UAL']"
1,0.0,JBLU,https://www.marketwatch.com/story/airline-stoc...,https://cdn.snapi.dev/images/v1/6/i/airline8.jpg,Airline stocks sink as analyst says United's m...,shares airline companies traded broadly lower ...,Negative,Market Watch,2020-08-31 13:58:53-04:00,Article,[],"['AAL', 'DAL', 'JBLU', 'LUV', 'UAL', 'JETS']"
2,0.0,JBLU,https://www.fool.com/investing/2020/08/27/sp-5...,https://cdn.snapi.dev/images/v1/y/z/urlhttps3a...,"S&P 500 Airline Stocks, Cruise Stocks Surge on...",big change fed policy good stocks 15 minute co...,Positive,The Motley Fool,2020-08-27 17:58:00-04:00,Article,[],"['AAL', 'CCL', 'DAL', 'JBLU', 'LUV', 'NCLH', '..."
3,0.0,JBLU,https://www.fool.com/investing/2020/08/27/why-...,https://cdn.snapi.dev/images/v1/b/w/urlhttps3a...,Why Airline Shares Are Higher Today (Aug 27),covid quick test could help reassure uncertain...,Positive,The Motley Fool,2020-08-27 13:25:00-04:00,Article,[],"['AAL', 'DAL', 'JBLU', 'LUV', 'SAVE', 'UAL']"
4,0.0,JBLU,https://www.zacks.com/stock/news/1048181/why-i...,https://cdn.snapi.dev/images/v1/0/m/airline7.jpg,Why Is JetBlue (JBLU) Up 12.1% Since Last Earn...,jetblue jblu reported earnings 30 days ago nex...,Positive,Zacks Investment Research,2020-08-27 11:31:26-04:00,Article,[],['JBLU']


In [14]:
df['train'] = np.random.rand(len(df)) < 0.8
df['id'] = list(range(len(df)))

In [15]:
term_breakup = pd.DataFrame()
for sentiment in list(set(df['sentiment'])):
  qwe = [x.split() for x in df.loc[(df['train'] == True) & (df['sentiment'] == sentiment), 'text']]
  qwe_flat = [] 
  for qwe_increment in qwe:
    qwe_flat.extend(qwe_increment)
  term_breakup_increment = pd.DataFrame({
      'term': qwe_flat
  })
  term_breakup_increment['sentiment'] = sentiment
  term_breakup = term_breakup.append(
    term_breakup_increment
  )
term_breakup = term_breakup.groupby(['term','sentiment'],  as_index = False, sort = False).size()
term_breakup = pd.DataFrame(term_breakup)
term_breakup = term_breakup.reset_index()
term_breakup = term_breakup.rename(columns = {0:'count'})
term_breakup = term_breakup.pivot(index='term', columns='sentiment', values='count')
term_breakup = term_breakup.reset_index()
term_breakup.loc[np.isnan(term_breakup['Positive']), 'Positive'] = 0
term_breakup.loc[np.isnan(term_breakup['Neutral']), 'Neutral'] = 0
term_breakup.loc[np.isnan(term_breakup['Negative']), 'Negative'] = 0
term_breakup['total'] = term_breakup['Negative'] + term_breakup['Positive'] + term_breakup['Neutral']
term_breakup = term_breakup.sort_values('total', ascending=False)
term_breakup['Positive'] = term_breakup['Positive'] / term_breakup['total']
term_breakup['Negative'] = term_breakup['Negative'] / term_breakup['total']
term_breakup['Neutral'] = term_breakup['Neutral'] / term_breakup['total']


In [16]:
train_features_names = [
  'Positive_sum_normalised','Positive_sum','Positive_mean','Positive_max','Positive_min',
  'Neutral_sum_normalised','Neutral_sum','Neutral_mean','Neutral_max','Neutral_min',
  'Negative_sum_normalised','Negative_sum','Negative_mean','Negative_max','Negative_min'
]
output_feature_name = 'sentiment'


In [17]:
def prepare_features(
  df
):

  term_breakup_train = pd.DataFrame()

  for sentiment in list(set(df['sentiment'])):
    qwe = [x.split() for x in df.loc[(df['sentiment'] == sentiment), 'text']]
    id = [x for x in df.loc[(df['sentiment'] == sentiment), 'id']]
    qwe_flat = [] 
    id_flat = [] 
    for qwe_increment_index in range(len(qwe)):
      qwe_increment = qwe[qwe_increment_index]
      qwe_flat.extend(qwe_increment)
      id_flat.extend([id[qwe_increment_index] for i in range(len(qwe_increment))])
    term_breakup_increment = pd.DataFrame({
        'term': qwe_flat,
        'id': id_flat
    })
    term_breakup_increment['sentiment'] = sentiment
    term_breakup_train = term_breakup_train.append(
      term_breakup_increment
    )

  term_breakup_train = pd.merge(
      term_breakup_train,
      term_breakup,
      on = 'term',
      how = 'left'
  )

  term_breakup_train = term_breakup_train.groupby(['id','sentiment'], as_index=False, sort = False).agg({
      'Positive': ['sum','mean','max','min'],
      'Neutral': ['sum','mean','max','min'],
      'Negative': ['sum','mean','max','min']
  })

  term_breakup_train.columns = term_breakup_train.columns.map('_'.join)

  for train_feature_name in train_features_names:
    if train_feature_name in term_breakup_train.columns:
      term_breakup_train.loc[np.isnan(term_breakup_train[train_feature_name]), train_feature_name] = 0

  term_breakup_train['Positive_sum_normalised'] = term_breakup_train['Positive_sum'] / ( term_breakup_train['Positive_sum'] + term_breakup_train['Neutral_sum'] + term_breakup_train['Negative_sum'] )
  term_breakup_train['Neutral_sum_normalised'] = term_breakup_train['Neutral_sum'] / ( term_breakup_train['Positive_sum'] + term_breakup_train['Neutral_sum'] + term_breakup_train['Negative_sum'] )
  term_breakup_train['Negative_sum_normalised'] = term_breakup_train['Negative_sum'] / ( term_breakup_train['Positive_sum'] + term_breakup_train['Neutral_sum'] + term_breakup_train['Negative_sum'] )

  for train_feature_name in train_features_names:
    term_breakup_train.loc[np.isnan(term_breakup_train[train_feature_name]), train_feature_name] = 0
  return term_breakup_train

term_features = prepare_features(df)

term_features = pd.merge(
    term_features,
    df[['id','train']],
    left_on = 'id_',
    right_on = 'id'
)

del term_features['id_']
term_features = term_features.rename(columns = {'sentiment_':'sentiment'})


In [38]:
clf = CatBoostClassifier(
    iterations=84,
    learning_rate=0.03
  )
clf.fit(
  term_features.loc[term_features['train'] == True, train_features_names], 
  term_features.loc[term_features['train'] == True, output_feature_name],
  eval_set=(term_features.loc[term_features['train'] == False, train_features_names], 
  term_features.loc[term_features['train'] == False, output_feature_name])
)
term_features['prediction'] = clf.predict(term_features[train_features_names])


0:	learn: 1.0752674	test: 1.0788264	best: 1.0788264 (0)	total: 173ms	remaining: 14.3s
1:	learn: 1.0538674	test: 1.0603833	best: 1.0603833 (1)	total: 349ms	remaining: 14.3s
2:	learn: 1.0334907	test: 1.0432594	best: 1.0432594 (2)	total: 537ms	remaining: 14.5s
3:	learn: 1.0147783	test: 1.0273436	best: 1.0273436 (3)	total: 731ms	remaining: 14.6s
4:	learn: 0.9971529	test: 1.0123360	best: 1.0123360 (4)	total: 919ms	remaining: 14.5s
5:	learn: 0.9810055	test: 0.9986382	best: 0.9986382 (5)	total: 1.1s	remaining: 14.3s
6:	learn: 0.9655397	test: 0.9857617	best: 0.9857617 (6)	total: 1.28s	remaining: 14.1s
7:	learn: 0.9512743	test: 0.9738868	best: 0.9738868 (7)	total: 1.45s	remaining: 13.8s
8:	learn: 0.9374445	test: 0.9624444	best: 0.9624444 (8)	total: 1.64s	remaining: 13.7s
9:	learn: 0.9244559	test: 0.9517725	best: 0.9517725 (9)	total: 1.82s	remaining: 13.5s
10:	learn: 0.9121914	test: 0.9418161	best: 0.9418161 (10)	total: 2s	remaining: 13.3s
11:	learn: 0.9007816	test: 0.9324586	best: 0.9324586 (11

In [42]:
print('Train accuracy:')
print(
    sum(term_features.loc[term_features['train'] == True,'prediction'] == term_features.loc[term_features['train'] == True,'sentiment']) / len(term_features.loc[term_features['train'] == True])
)

print('Test accuracy:')
print(
    sum(term_features.loc[term_features['train'] == False,'prediction'] == term_features.loc[term_features['train'] == False,'sentiment']) / len(term_features.loc[term_features['train'] == False])
)

print('Test confusion matrix:')
pd.crosstab(
  term_features.loc[term_features['train'] == False,'sentiment'],
  term_features.loc[term_features['train'] == False,'prediction'],
  rownames=['Actual'], 
  colnames=['Predicted']
)


Train accuracy:
0.7114989512567023
Test accuracy:
0.6645703504659539
Test confusion matrix:


Predicted,Negative,Neutral,Positive
Actual,,,
Negative,7156,2000,3711
Neutral,2597,12720,8505
Positive,2275,4380,26620
